In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import sys
from absl import app
from absl import flags
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()

import efficientnet_builder
import preprocessing
from sklearn.metrics import roc_auc_score
import time

In [9]:
import pandas as pd
import numpy as np

def get_data():
  path = './../main/'
  train = pd.read_csv(path+'CheXpert-v1.0-small/train.csv')
  valid = pd.read_csv(path+'CheXpert-v1.0-small/valid.csv')
  
  train['validation'] = False
  valid['validation'] = True
  df = pd.concat([train, valid])
  
  columns = ['Path', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'validation']
  df = df[columns]
  
  for feature in ['Atelectasis', 'Edema']:
      df[feature] = df[feature].apply(lambda x: 1 if x==-1 else x)
  
  for feature in ['Cardiomegaly', 'Consolidation', 'Pleural Effusion']:
      df[feature] = df[feature].apply(lambda x: 0 if x==-1 else x)
  df.fillna(0, inplace=True)
  
  train = df[~df.validation][:500]
  print(len(train))
  train_files = train['Path'].tolist()
  train_files = [path+fil for fil in train_files]
  
  columns = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
  train_labels = np.array(train[columns])
  
  valid = df[df.validation]#[:50]
  print(len(valid))
  valid_files = valid['Path'].tolist()
  valid_files = [path+fil for fil in valid_files]
  
  columns = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
  valid_labels = np.array(valid[columns])  
  return train_files, train_labels, valid_files, valid_labels

In [10]:
MEAN_RGB = [0.485 * 255, 0.456 * 255, 0.406 * 255]
STDDEV_RGB = [0.229 * 255, 0.224 * 255, 0.225 * 255]


model_name='efficientnet-b4'
batch_size=32
"""Initialize internal variables."""
model_name = model_name
batch_size = batch_size
num_classes = 1000
# Model Scaling parameters
_, _, image_size, _ = efficientnet_builder.efficientnet_params(
      model_name)
print('image_size', image_size)

def restore_model(sess, ckpt_dir):
  """Restore variables from checkpoint dir."""
  checkpoint = tf.train.latest_checkpoint(ckpt_dir)
  ema = tf.train.ExponentialMovingAverage(decay=0.9999)
  ema_vars = tf.trainable_variables() + tf.get_collection('moving_vars')
  for v in tf.global_variables():
    if 'moving_mean' in v.name or 'moving_variance' in v.name:
      ema_vars.append(v)
  ema_vars = list(set(ema_vars))
  var_dict = ema.variables_to_restore(ema_vars)
  saver = tf.train.Saver(var_dict, max_to_keep=1)
  saver.restore(sess, checkpoint)
  return saver

def build_model(features, is_training):
  """Build model with input features."""
  features -= tf.constant(MEAN_RGB, shape=[1, 1, 3], dtype=features.dtype)
  features /= tf.constant(STDDEV_RGB, shape=[1, 1, 3], dtype=features.dtype)
  out, _ = efficientnet_builder.build_model_base(
      features, model_name, is_training)
  return out

def build_dataset(filenames, labels, is_training):
  """Build input dataset."""
  filenames = tf.constant(filenames)
  labels = tf.constant(labels)
  dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

  def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = preprocessing.preprocess_image(
        image_string, is_training, image_size=image_size)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

  dataset = dataset.map(_parse_function)
  dataset = dataset.batch(batch_size)#.repeat()
  return dataset



def _loss(x, y):
  logits = tf.contrib.layers.fully_connected(x, 5, activation_fn=None)  
  predicts = tf.math.sigmoid(logits, name = 'sigmoid_logits')
  cross_entropy = tf.losses.sigmoid_cross_entropy(logits=logits,
                                                  multi_class_labels=y)
  weight_decay = 1e-5
  loss = cross_entropy + weight_decay * tf.add_n(
      [tf.nn.l2_loss(v) for v in tf.trainable_variables()
       if 'batch_normalization' not in v.name])
  return loss, predicts

image_size 224


In [11]:
def calculate_metric(predictions, validation_labels):
  labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
  auc_scores = {labels[i]: roc_auc_score(validation_labels[:predictions.shape[0],i],predictions[:,i]) for i in range(5)}
  chexpert_auc_scores = {
                         'Atelectasis':      0.858,
                         'Cardiomegaly':     0.854,
                         'Consolidation':    0.939,
                         'Edema':            0.941,
                         'Pleural Effusion': 0.936
                        }
  max_feat_len = max(map(len, labels))
#   if epoch+1 == epochs:
#     [print(f'{k: <{max_feat_len}}\t auc: {auc_scores[k]:.3}\t chexpert auc: {chexpert_auc_scores[k]:.3}\t difference: {(chexpert_auc_scores[k]-auc_scores[k]):.3}') for k in labels]
  
  avg_chexpert_auc = sum(list(chexpert_auc_scores.values()))/len(chexpert_auc_scores.values())
  avg_auc          = sum(list(auc_scores.values()))/len(auc_scores.values())
  
  print('Average auc: {} \t CheXpert average auc {}'.format(avg_auc, avg_chexpert_auc))
  return

In [12]:
def train(lr = 0.001, epochs=3, ckpt_dir='./weights_{}/'.format(model_name)):
  graph = tf.Graph()
  with tf.Session(graph=graph) as sess:
    train_files, train_labels, validation_files, validation_labels = get_data()
    train_dataset = build_dataset(train_files, train_labels, True)
    valid_dataset = build_dataset(validation_files, validation_labels, False)
    
    train_iter = train_dataset.make_initializable_iterator()
    valid_iter = valid_dataset.make_initializable_iterator()
    
    train_images, train_labels = train_iter.get_next()
    valid_images, valid_labels = valid_iter.get_next()
  
    train_out = build_model(train_images, is_training=True)
    train_out = tf.reduce_mean(train_out, axis=[1,2])
  
  
    with tf.variable_scope(tf.get_variable_scope(), reuse=True):
      valid_out = build_model(valid_images, is_training=False)
      valid_out = tf.reduce_mean(valid_out, axis=[1,2])
    
    sess.run(tf.global_variables_initializer())
    saver = restore_model(sess, ckpt_dir)
  
    temp = set(tf.all_variables())
  
    train_loss, _ = _loss(train_out, train_labels)
    valid_loss, predicts = _loss(valid_out, valid_labels)
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = lr
    learning_rate = tf.compat.v1.train.exponential_decay(starter_learning_rate,
                                                         global_step,
                                                         100000, 0.96, 
                                                         staircase=True)

    optimizer = tf.train.MomentumOptimizer(
        learning_rate=learning_rate, momentum=0.9, name="momentum_opt").minimize(train_loss, global_step=global_step)
      
    sess.run(tf.initialize_variables(set(tf.all_variables()) - temp))
    
    saver = tf.train.Saver()
#     saver.restore(sess, './saved_model/model_efficientnet-b4_0.761')
    
    n_batches_train = len(train_files)//batch_size
    n_batches_valid = len(validation_files)//batch_size
    
    print(n_batches_train, n_batches_valid)
    print('training starts')
    for epoch in range(epochs):
      tic = time.time()
      # training
      sess.run(train_iter.initializer)
      _train_loss = 0
      for i in range(n_batches_train):
        loss_value, _ = sess.run([train_loss, optimizer])
        if i>1 and i%100==0:
          print('iteration: {} | loss {}'.format(i, loss_value))
        _train_loss += loss_value
      _train_loss = _train_loss/n_batches_train
      
      # validation
      sess.run(valid_iter.initializer)
      _valid_loss = 0
      predictions = []
      for i in range(n_batches_valid+1):
        loss_value, prediction = sess.run([valid_loss, predicts])
        _valid_loss += loss_value
        predictions.append(prediction)
      _valid_loss = _valid_loss/n_batches_valid
      calculate_metric(np.vstack(predictions), validation_labels)
      print('elapsed time: {}'.format(time.time()-tic))
      print("epoch: {} | training loss: {} | validation loss: {}".format(epoch+1, _train_loss, _valid_loss ))
      saver.save(sess, "./saved_model/model_{}_{}".format(model_name, np.round(_valid_loss, 3)))

In [ ]:
train(lr = 0.001, epochs=2)

500
234


In [ ]:
1157/60